# Collecting Data from the Spotify Web API using Spotipy

## About the Spotipy Library:

From the [official Spotipy docs](https://spotipy.readthedocs.io/en/latest/): 
>"Spotipy is a lightweight Python library for the Spotify Web API. With Spotipy you get full access to all of the music data provided by the Spotify platform."


## About using the Spotify Web API:

Spotify offers a number of [API endpoints](https://beta.developer.spotify.com/documentation/web-api/reference/) to access the Spotify data. In this notebook, I used the following:

- [search endpoint](https://beta.developer.spotify.com/documentation/web-api/reference/search/search/) to get the track IDs 
- [audio features endpoint](https://beta.developer.spotify.com/documentation/web-api/reference/tracks/get-several-audio-features/) to get the corresponding audio features.

The data was collected on several days during the months of April, May and August 2018.


## Goal of this notebook:

The goal is to show how to collect audio features data for tracks from the [official Spotify Web API](https://beta.developer.spotify.com/documentation/web-api/) in order to use it for further analysis/ machine learning which will be part of another notebook.

# 1. Setting Up

The below code is sufficient to set up Spotipy for querying the API endpoint. A more detailed explanation of the whole procedure is available in the [official docs](https://spotipy.readthedocs.io/en/latest/#installation).

In [4]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid ="xx" 
secret = "xx"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# 2. Get the Track ID Data

The data collection is divided into 2 parts: the track IDs and the audio features. In this step, I'm going to collect 10.000 track IDs from the Spotify API.

The [search endpoint](https://beta.developer.spotify.com/documentation/web-api/reference/search/search/) used in this step had a few limitations:

- limit: a maximum of 50 results can be returned per query
- offset: this is the index of the first result to return, so if you want to get the results with the index 50-100 you will need to set the offset to 50 etc.

Spotify cut down the maximum offset to 10.000 (as of May 2018?), I was lucky enough to do my first collection attempt while it was still 100.000

My solution: using a nested for loop, I increased the offset by 50 in the outer loop until the maxium limit/ offset was reached. The inner for loop did the actual querying and appending the returned results to appropriate lists which I used afterwards to create my dataframe.

In [14]:
# timeit library to measure the time needed to run this code
import timeit
start = timeit.default_timer()

# create empty lists where the results are going to be stored
artist_name = []
track_name = []
popularity = []
track_id = []

for i in range(0,5000,50):
    track_results = sp.search(q='year:2018', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])
      

stop = timeit.default_timer()
print ('Time to run this code (in seconds):', stop - start)

Time to run this code (in seconds): 25.634927900000037


In [34]:
import timeit
start = timeit.default_timer()
track_results = sp.search(q="year:2018", type="track", limit=50, offset=0)
stop = timeit.default_timer()
print('Time to run this code (in seconds):', stop - start )

Time to run this code (in seconds): 0.12312669999994341


In [46]:
tracks = track_results['tracks']
for i in tracks:
    print(i)
print()

print(tracks['items'])

href
items
limit
next
offset
previous
total

[{'album': {'album_type': 'single', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7uaIm6Pw7xplS8Dy06V6pT'}, 'href': 'https://api.spotify.com/v1/artists/7uaIm6Pw7xplS8Dy06V6pT', 'id': '7uaIm6Pw7xplS8Dy06V6pT', 'name': 'Trevor Daniel', 'type': 'artist', 'uri': 'spotify:artist:7uaIm6Pw7xplS8Dy06V6pT'}], 'available_markets': ['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH', 'BO', 'BR', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DE', 'DK', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FR', 'GB', 'GR', 'GT', 'HK', 'HN', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JO', 'JP', 'KW', 'LB', 'LI', 'LT', 'LU', 'LV', 'MA', 'MC', 'MT', 'MX', 'MY', 'NI', 'NL', 'NO', 'NZ', 'OM', 'PA', 'PE', 'PH', 'PL', 'PS', 'PT', 'PY', 'QA', 'RO', 'SA', 'SE', 'SG', 'SK', 'SV', 'TH', 'TN', 'TR', 'TW', 'US', 'UY', 'VN', 'ZA'], 'external_urls': {'spotify': 'https://open.spotify.com/album/1Czfd5tEby3DbdYNdqzrCa'}, 'href': 'https://api.spotify.com/v1/albu

In [58]:
# for i in range(0,5000,50):
#     track_results = sp.search(q='year:2018', type='track', limit=50,offset=i)
#     for i, t in enumerate(track_results['tracks']['items']):
#         artist_name.append(t['artists'][0]['name'])
#         track_name.append(t['name'])
#         track_id.append(t['id'])
#         popularity.append(t['popularity'])

pairs = []
for i in range(0, 5000, 50):
    artist_results = sp.search(q = "aaaa", type="artist", limit=50, offset=i)
    for j, t in enumerate(artist_results['artists']['items']):
        new_pair = (t['name'], t['id'])
        pairs.append(new_pair)
    old_request = artist_results

In [72]:
ar = sp.search(q = "a", type="artist", offset = 0, limit=50)
print(ar['artists']['total'])

10020


# 3. EDA + Data Preparation

In the next few cells, I'm going to do some exploratory data analysis as well as data preparation of the newly gained data.

A quick check for the track_id list:

In [15]:
print('number of elements in the track_id list:', len(track_id))

number of elements in the track_id list: 5000


Looks good. Now loading the lists in a dataframe.

In [16]:
import pandas as pd

df_tracks = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity})
print(df_tracks.shape)
df_tracks.head()

(5000, 4)


,artist_name,track_name,track_id,popularity
0,Trevor Daniel,Falling,4TnjEaWOeW0eKTKIEvJyCa,97
1,Juice WRLD,Lucid Dreams,285pBltuF7vW8TeWk8hdRR,91
2,Juice WRLD,All Girls Are The Same,4VXIryQMWpIdGgYR4TrjT1,85
3,Post Malone,Sunflower - Spider-Man: Into the Spider-Verse,3KkXRkHbMCARz0aVfEt68P,87
4,Billie Eilish,lovely (with Khalid),0u2P5u6lvoDfwTYjAADbn4,89


In [17]:
df_tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 4 columns):
artist_name    5000 non-null object
track_name     5000 non-null object
track_id       5000 non-null object
popularity     5000 non-null int64
dtypes: int64(1), object(3)
memory usage: 156.4+ KB


Sometimes, the same track is returned under different track IDs (single, as part of an album etc.).

This needs to be checked for and corrected if needed.

In [18]:
# group the entries by artist_name and track_name and check for duplicates

grouped = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped[grouped > 1].count()

150

There are 524 duplicate entries which will be dropped in the next cell:

In [19]:
df_tracks.drop_duplicates(subset=['artist_name','track_name'], inplace=True)

In [20]:
# doing the same grouping as before to verify the solution
grouped_after_dropping = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped_after_dropping[grouped_after_dropping > 1].count()

0

This time the results are empty. Another way of checking this:

In [21]:
df_tracks[df_tracks.duplicated(subset=['artist_name','track_name'],keep=False)].count()

artist_name    0
track_name     0
track_id       0
popularity     0
dtype: int64

Checking how many tracks are left now:

In [22]:
df_tracks.shape

(4848, 4)

# 4: Get the Audio Features Data

With the [audio features endpoint](https://beta.developer.spotify.com/documentation/web-api/reference/tracks/get-several-audio-features/) I will now get the audio features data for my 9460 track IDs.

The limitation for this endpoint is that a maximum of 100 track IDs can be submitted per query.

Again, I used a nested for loop. This time the outer loop was pulling track IDs in batches of size 100 and the inner for loop was doing the query and appending the results to the rows list.

Additionaly, I had to implement a check when a track ID didn't return any audio features (i.e. None was returned) as this was causing issues.

In [23]:
# again measuring the time
start = timeit.default_timer()

# empty list, batchsize and the counter for None results
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(df_tracks['track_id']),batchsize):
    batch = df_tracks['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks where no audio features were available:',None_counter)

stop = timeit.default_timer()
print ('Time to run this code (in seconds):',stop - start)

Number of tracks where no audio features were available: 0
Time to run this code (in seconds): 2.8649849000000245


# 5. EDA + Data Preparation

Same as with the first dataset, checking how the rows list looks like:

In [24]:
print('number of elements in the track_id list:', len(rows))

number of elements in the track_id list: 4848


Finally, I will load the audio features in a dataframe.

In [25]:
df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.head()

Shape of the dataset: (4848, 18)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.784,0.430,10,-8.756,0,0.0364,0.1230,0.000000,0.0887,0.236,127.087,audio_features,4TnjEaWOeW0eKTKIEvJyCa,spotify:track:4TnjEaWOeW0eKTKIEvJyCa,https://api.spotify.com/v1/tracks/4TnjEaWOeW0e...,https://api.spotify.com/v1/audio-analysis/4Tnj...,159382,4
1,0.511,0.566,6,-7.230,0,0.2000,0.3490,0.000000,0.3400,0.218,83.903,audio_features,285pBltuF7vW8TeWk8hdRR,spotify:track:285pBltuF7vW8TeWk8hdRR,https://api.spotify.com/v1/tracks/285pBltuF7vW...,https://api.spotify.com/v1/audio-analysis/285p...,239836,4
2,0.671,0.529,0,-7.226,1,0.3070,0.0769,0.000335,0.0856,0.203,161.989,audio_features,4VXIryQMWpIdGgYR4TrjT1,spotify:track:4VXIryQMWpIdGgYR4TrjT1,https://api.spotify.com/v1/tracks/4VXIryQMWpId...,https://api.spotify.com/v1/audio-analysis/4VXI...,165820,4
3,0.760,0.479,2,-5.574,1,0.0466,0.5560,0.000000,0.0703,0.913,89.911,audio_features,3KkXRkHbMCARz0aVfEt68P,spotify:track:3KkXRkHbMCARz0aVfEt68P,https://api.spotify.com/v1/tracks/3KkXRkHbMCAR...,https://api.spotify.com/v1/audio-analysis/3KkX...,158040,4
4,0.351,0.296,4,-10.109,0,0.0333,0.9340,0.000000,0.0950,0.120,115.284,audio_features,0u2P5u6lvoDfwTYjAADbn4,spotify:track:0u2P5u6lvoDfwTYjAADbn4,https://api.spotify.com/v1/tracks/0u2P5u6lvoDf...,https://api.spotify.com/v1/audio-analysis/0u2P...,200186,4


In [26]:
df_audio_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4848 entries, 0 to 4847
Data columns (total 18 columns):
danceability        4848 non-null float64
energy              4848 non-null float64
key                 4848 non-null int64
loudness            4848 non-null float64
mode                4848 non-null int64
speechiness         4848 non-null float64
acousticness        4848 non-null float64
instrumentalness    4848 non-null float64
liveness            4848 non-null float64
valence             4848 non-null float64
tempo               4848 non-null float64
type                4848 non-null object
id                  4848 non-null object
uri                 4848 non-null object
track_href          4848 non-null object
analysis_url        4848 non-null object
duration_ms         4848 non-null int64
time_signature      4848 non-null int64
dtypes: float64(9), int64(4), object(5)
memory usage: 681.9+ KB


Some columns are not needed for the analysis so I will drop them.

Also the ID column will be renamed to track_id so that it matches the column name from the first dataframe.

In [27]:
columns_to_drop = ['analysis_url','track_href','type','uri']
df_audio_features.drop(columns_to_drop, axis=1,inplace=True)

df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)

df_audio_features.shape

(4848, 14)

In [28]:
# merge both dataframes
# the 'inner' method will make sure that we only keep track IDs present in both datasets
df = pd.merge(df_tracks,df_audio_features,on='track_id',how='inner')
print("Shape of the dataset:", df_audio_features.shape)
df.head()

Shape of the dataset: (4848, 14)


,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Trevor Daniel,Falling,4TnjEaWOeW0eKTKIEvJyCa,97,0.784,0.430,10,-8.756,0,0.0364,0.1230,0.000000,0.0887,0.236,127.087,159382,4
1,Juice WRLD,Lucid Dreams,285pBltuF7vW8TeWk8hdRR,91,0.511,0.566,6,-7.230,0,0.2000,0.3490,0.000000,0.3400,0.218,83.903,239836,4
2,Juice WRLD,All Girls Are The Same,4VXIryQMWpIdGgYR4TrjT1,85,0.671,0.529,0,-7.226,1,0.3070,0.0769,0.000335,0.0856,0.203,161.989,165820,4
3,Post Malone,Sunflower - Spider-Man: Into the Spider-Verse,3KkXRkHbMCARz0aVfEt68P,87,0.760,0.479,2,-5.574,1,0.0466,0.5560,0.000000,0.0703,0.913,89.911,158040,4
4,Billie Eilish,lovely (with Khalid),0u2P5u6lvoDfwTYjAADbn4,89,0.351,0.296,4,-10.109,0,0.0333,0.9340,0.000000,0.0950,0.120,115.284,200186,4


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4848 entries, 0 to 4847
Data columns (total 17 columns):
artist_name         4848 non-null object
track_name          4848 non-null object
track_id            4848 non-null object
popularity          4848 non-null int64
danceability        4848 non-null float64
energy              4848 non-null float64
key                 4848 non-null int64
loudness            4848 non-null float64
mode                4848 non-null int64
speechiness         4848 non-null float64
acousticness        4848 non-null float64
instrumentalness    4848 non-null float64
liveness            4848 non-null float64
valence             4848 non-null float64
tempo               4848 non-null float64
duration_ms         4848 non-null int64
time_signature      4848 non-null int64
dtypes: float64(9), int64(5), object(3)
memory usage: 681.8+ KB


Just in case, checking for any duplicate tracks:

In [30]:
df[df.duplicated(subset=['artist_name','track_name'],keep=False)]

,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature


Everything seems to be fine so I will save the dataframe as a .csv file.

In [31]:
df.to_csv('SpotifyAudioFeatures08082018.csv')